In [2]:
#importing all the libraries
from pathlib import Path
import re
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
from seqeval.metrics import f1_score,accuracy_score
from transformers import Trainer, TrainingArguments,DistilBertForTokenClassification,DistilBertTokenizerFast

In [3]:
import torch
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
def encode_tags(tag2id,tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100 
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

def preprocess(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)
    texts=token_docs
    tags=tag_docs
    unique_tags = set(tag for doc in tags for tag in doc)

    tag2id = {tag: id for id, tag in enumerate(unique_tags)}

    train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

    train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

    train_labels = encode_tags(tag2id,train_tags, train_encodings)
    val_labels = encode_tags(tag2id,val_tags, val_encodings)

    train_encodings.pop("offset_mapping") # we don't want to pass this to the model
    val_encodings.pop("offset_mapping")
    train_dataset = Dataset(train_encodings, train_labels)
    val_dataset = Dataset(val_encodings, val_labels)

    return train_dataset,val_dataset,unique_tags

def Schedular(schedular_name="PB2"):
    if(schedular_name=="PB2"):
    from ray.tune.schedulers.pb2 import PB2
    scheduler = PB2(
            metric='eval_f1',
            mode='max',
            hyperparam_bounds={
                "learning_rate": [1e-5, 1e-3]   }
                )
    return scheduler
  else:
    from ray.tune.schedulers import PopulationBasedTraining
    scheduler = PopulationBasedTraining(
        metric="eval_acc",
        mode="max",
        hyperparam_mutations={
            "learning_rate": [1e-5, 5e-5]
        })
    return scheduler

def model_init():
    return DistilBertForTokenClassification.from_pretrained("distilbert-base-cased",num_labels=len(unique_tags))

def my_compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    label_list=list(unique_tags)
    true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
    true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
    return {'f1':f1_score(true_labels,true_predictions),
            'acc':accuracy_score(true_labels,true_predictions)}

train_dataset,val_dataset,unique_tags = preprocess('C:/Users/m1061059/Downloads/wnut17train.conll')


training_args = TrainingArguments(
        output_dir=".",
        learning_rate=1e-5,  # config
        do_train=True,
        do_eval=True,
        evaluation_strategy="epoch", #no and steps
        load_best_model_at_end=True,
        num_train_epochs=1,  # config
        max_steps=-1,
        per_device_train_batch_size=16,  # config
        per_device_eval_batch_size=16,  # config
        warmup_steps=0,
        weight_decay=0.1,  # config
        logging_dir="./logs",
    )


trainer = Trainer(
    model_init=model_init,                        
    args=training_args,                 
    train_dataset=train_dataset,        
    eval_dataset=val_dataset   ,
    compute_metrics=my_compute_metrics, 
)

trainer.train()

trainer.evaluate()

schedular=Schedular()

trainer.hyperparameter_search(n_trials=1,direction="maximize",backend="ray",scheduler=schedular)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch,Training Loss,Validation Loss,F1,Acc,Runtime,Samples Per Second
1,No log,0.194556,0.005495,0.955554,76.978500,8.821000


Attempted to log scalar metric eval_loss:
0.19455580413341522
Attempted to log scalar metric eval_f1:
0.005494505494505495
Attempted to log scalar metric eval_acc:
0.9555537823172678
Attempted to log scalar metric eval_runtime:
76.9785
Attempted to log scalar metric eval_samples_per_second:
8.821
Attempted to log scalar metric epoch:
1.0
Attempted to log scalar metric train_runtime:
1397.202
Attempted to log scalar metric train_samples_per_second:
0.122
Attempted to log scalar metric total_flos:
95591052684900
Attempted to log scalar metric epoch:
1.0


Attempted to log scalar metric eval_loss:
0.19455580413341522
Attempted to log scalar metric eval_f1:
0.005494505494505495
Attempted to log scalar metric eval_acc:
0.9555537823172678
Attempted to log scalar metric eval_runtime:
77.1575
Attempted to log scalar metric eval_samples_per_second:
8.8
Attempted to log scalar metric epoch:
1.0
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102550 sha256=fd0ac11e44240febd22284c398ac372eb1994f2ada4e14198b284f29dd6e8168
  Stored in directory: c:\users\m1061059\appdata\local\pip\cache\wheels\72\d5\80\7921b405df363b0f10446fc93491092c20bc75491cbbd0a354
Successfully built paramz


You are using PopulationBasedTraining but you haven't enabled checkpointing. This means your trials will train from scratch everytime they are exploiting new configurations. Consider enabling checkpointing by passing `keep_checkpoints_num=1` as an additional argument to `Trainer.hyperparameter_search`.
2021-03-29 15:08:31,500	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-29 15:08:47,458	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-03-29 15:08:47,724	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


...

(pid=30636) <IPython.core.display.HTML object>
Result for _inner_8f976_00000:
  date: 2021-03-29_16-13-53
  done: false
  eval_acc: 0.9549952122566231
  eval_f1: 0.44054580896686163
  eval_loss: 0.2143019437789917
  eval_runtime: 133.2084
  eval_samples_per_second: 5.097
  experiment_id: 1d4f08c6c03f4a21ba5f47045d5e4d6b
  hostname: A2ML41525
  iterations_since_restore: 2
  node_ip: 192.168.0.102
  objective: 139.7009410212235
  pid: 30636
  time_since_restore: 3904.5434544086456
  time_this_iter_s: 2024.8206660747528
  time_total_s: 3904.5434544086456
  timestamp: 1617014633
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: 8f976_00000
  
== Status ==
Memory usage on this node: 11.7/15.8 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1/8 CPUs, 0/0 GPUs, 0.0/3.42 GiB heap, 0.0/1.17 GiB objects
Result logdir: C:\Users\m1061059\ray_results\_inner_2021-03-29_15-08-47
Number of trials: 1/1 (1 RUNNING)
+--------------------+----------+-------------

2021-03-29 16:13:59,710	WARNING tune.py:429 -- Trial Runner checkpointing failed: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\m1061059\\ray_results\\_inner_2021-03-29_15-08-47\\.tmp_generator' -> 'C:\\Users\\m1061059\\ray_results\\_inner_2021-03-29_15-08-47\\basic-variant-state-2021-03-29_15-08-47.json'
2021-03-29 16:13:59,713	INFO tune.py:450 -- Total run time: 3929.52 seconds (3912.02 seconds for the tuning loop).


Result for _inner_8f976_00000:
  date: 2021-03-29_16-13-53
  done: true
  eval_acc: 0.9549952122566231
  eval_f1: 0.44054580896686163
  eval_loss: 0.2143019437789917
  eval_runtime: 133.2084
  eval_samples_per_second: 5.097
  experiment_id: 1d4f08c6c03f4a21ba5f47045d5e4d6b
  experiment_tag: 0_learning_rate=5.6115e-06,num_train_epochs=2,per_device_train_batch_size=32,seed=38.078
  hostname: A2ML41525
  iterations_since_restore: 2
  node_ip: 192.168.0.102
  objective: 139.7009410212235
  pid: 30636
  time_since_restore: 3904.5434544086456
  time_this_iter_s: 2024.8206660747528
  time_total_s: 3904.5434544086456
  timestamp: 1617014633
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: 8f976_00000
  
== Status ==
Memory usage on this node: 12.4/15.8 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/3.42 GiB heap, 0.0/1.17 GiB objects
Result logdir: C:\Users\m1061059\ray_results\_inner_2021-03-29_15-08-47
Number of trials: 1/1

BestRun(run_id='8f976_00000', objective=139.7009410212235, hyperparameters={'learning_rate': 5.61151641533451e-06, 'num_train_epochs': 2, 'seed': 38.07785794998673, 'per_device_train_batch_size': 32})

2021-03-29 20:55:21,623	ERROR worker.py:1109 -- listen_error_messages_raylet: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-03-29 20:55:21,650	ERROR import_thread.py:89 -- ImportThread: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
2021-03-29 20:55:21,978	ERROR worker.py:919 -- print_logs: Error while reading from socket: (10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
